In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import matplotlib.pyplot as plt
import seaborn as sns

base_model_name = "HuggingFaceTB/SmolLM2-360M"
chat_model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"
noah_model_name = "nkasmanoff/picard"

model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
     torch_dtype=torch.bfloat16,
   device_map='auto'
)

base_model = AutoModelForCausalLM.from_pretrained(base_model_name, **model_kwargs)
chat_model = AutoModelForCausalLM.from_pretrained(chat_model_name, **model_kwargs)
noah_model = AutoModelForCausalLM.from_pretrained(noah_model_name, **model_kwargs)


checkpoint_path = "HuggingFaceTB/SmolLM2-360M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.model_max_length = 2048
tokenizer.pad_token = "<|endoftext|>"  # note this is specific to smollm
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token )
tokenizer.padding_side = 'right'


In [2]:


def get_residuals(base_model, chat_model, final_model):
    """
    For every parameter in this model, calculate the residual between base_model and chat_model. 
    Add that residual value to final_model.
    """

    for base_param, chat_param, final_param in zip(base_model.parameters(), chat_model.parameters(), final_model.parameters()):
        residual = chat_param - base_param
        final_param.data += 0.05 * residual

    return final_model

def average_weights(chat_model,final_model):
    """For every parameter in this model, calculate the average between chat_model and final_model.
    """

    for chat_param, final_param in zip(chat_model.parameters(), final_model.parameters()):
        avg = (chat_param + final_param) / 2
        final_param.data = avg

    return final_model
    
final_model = get_residuals(base_model, chat_model, noah_model)
#inal_model = average_weights(chat_model, noah_model)

In [12]:
system_prompt ="<|im_start|>system\nYou are Pi-Card.<|im_end|>\n"

#system_prompt = ""
prompt = """I have 45 pills. Sofie dose is 1 pill in morning and half pill at night. How long will this last"""
#prompt = "What does your name stand for?"
#prompt = "Who is Sofie?"
#prompt = "Who is your favorite cat"
#prompt = "What is the square root of 144?"
#prompt = "Who are you?"
#prompt = "What is the oort cloud? Explain in 3 sentences"
prompt = f"{system_prompt}<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"


In [13]:
model = final_model
model.eval();

input_ids = tokenizer.encode(prompt, return_tensors='pt')
input_ids = input_ids.to(model.device)
output = model.generate(input_ids, max_new_tokens=512,  do_sample=False, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False, pad_token_id = tokenizer.eos_token_id)
formatted_output_text = "<|im_end|>".join(output_text.split("<|im_end|>")[:3]) + "<|im_end|>"
print(formatted_output_text)

<|im_start|>system
You are Pi-Card.<|im_end|>
<|im_start|>user
I have 45 pills. Sofie dose is 1 pill in morning and half pill at night. How long will this last<|im_end|>
<|im_start|>assistant
If Sofie's pill regimen is typical, with a dose of 1 pill in the morning and a half-dose at night, the total dose is 45 pills. Assuming a consistent daily routine, the medication should take about 3 days to wear off, as each pill is responsible for one-third of the total dose.<|im_end|>


In [15]:
# Save the model
save_path = "picard-merged"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Saved model to {save_path}")

# Create GGUF model
!python llama.cpp/convert_hf_to_gguf.py picard-merged --outfile picard-merged.gguf --outtype f16
print(f"Saved GGUF model to picard-merged.gguf")

Saved model to picard-merged


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO:hf-to-gguf:Loading model: picard-merged
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {960, 49152}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {960}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> F16, shape = {2560, 960}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> F16, shape = {960, 2560}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> F16, shape = {960, 2560}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {960}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfloat16 --> F16, shape = {960, 320}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.bfloat16 --> F16, shape = {960, 960}
INFO:hf-to-gguf:blk.0.attn_q.weight,         torch.bfloat16 --> F16, shape = {960, 960}
INF

In [ ]:
chat_model = AutoModelForCausalLM.from_pretrained(chat_model_name, **model_kwargs)

model = chat_model
model.eval();

input_ids = tokenizer.encode(prompt, return_tensors='pt')
input_ids = input_ids.to(model.device)
output = model.generate(input_ids, max_new_tokens=256,  do_sample=False, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False, pad_token_id = tokenizer.eos_token_id)
formatted_output_text = "<|im_end|>".join(output_text.split("<|im_end|>")[:3]) + "<|im_end|>"
print(formatted_output_text)

In [ ]:
#chat_model = AutoModelForCausalLM.from_pretrained(chat_model_name, **model_kwargs)
noah_model = AutoModelForCausalLM.from_pretrained(noah_model_name, **model_kwargs)
model = noah_model
model.eval();


input_ids = tokenizer.encode(prompt, return_tensors='pt')
input_ids = input_ids.to(model.device)
output = model.generate(input_ids, max_new_tokens=256,  do_sample=False, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False, pad_token_id = tokenizer.eos_token_id)
formatted_output_text = "<|im_end|>".join(output_text.split("<|im_end|>")[:3]) + "<|im_end|>"
print(formatted_output_text)

In [ ]:
print(output_text)

In [ ]:
chat_model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(chat_model_name)
tokenizer.model_max_length = 2048
tokenizer.pad_token = "<|endoftext|>"  # note this is specific to smollm
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token )
tokenizer.padding_side = 'right'

chat_model = AutoModelForCausalLM.from_pretrained(chat_model_name, **model_kwargs)

model = chat_model
model.eval();

input_ids = tokenizer.encode(prompt, return_tensors='pt')
input_ids = input_ids.to(model.device)
output = model.generate(input_ids, max_new_tokens=256,  do_sample=False, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False, pad_token_id = tokenizer.eos_token_id)
formatted_output_text = "<|im_end|>".join(output_text.split("<|im_end|>")[:3]) + "<|im_end|>"
print(formatted_output_text)